In [1]:
import fitbit
import datetime
import pandas as pd
import json
import numpy as np

In [3]:
def convert_string_to_date(date):
    return datetime.datetime.strptime(date, '%m/%d/%Y').date()

In [4]:
def days_between(d1, d2):
    return abs((d2 - d1).days)

In [5]:
def get_daily_data(fitbit_object,activity_type,ID,start_date,last_date,treatment,u_ID):
    fitbit_data=fitbit_object.time_series(activity_type, user_id=ID, base_date=start_date,end_date=last_date)
    activity_type=activity_type.replace('/', "-")
    dfItem=pd.io.json.json_normalize(fitbit_data[activity_type])
    dfItem['group']=treatment
    dfItem['user_ID']=u_ID
    dfItem['month_of_treatment']=(np.arange(days_between(start_date,last_date)+1)/30).astype(int)
    dfItem['day_of_treatment']=np.arange(days_between(start_date,last_date)+1)
    return dfItem

In [7]:
df_calories_till_today=pd.DataFrame()
df_steps_till_today=pd.DataFrame()
df_distance_till_today=pd.DataFrame()
df_bmi_till_today=pd.DataFrame()
df_fat_till_today=pd.DataFrame()
df_weight_till_today=pd.DataFrame()
df_minutesSedentary_till_today=pd.DataFrame()
df_minutesLightlyActive_till_today=pd.DataFrame()
df_minutesFairlyActive_till_today=pd.DataFrame()
df_minutesVeryActive_till_todaye=pd.DataFrame()


activity =["calories", "steps", "distance","minutesSedentary","minutesLightlyActive","minutesFairlyActive","minutesVeryActive","activityCalories","bmi","fat","weight"]

fitbit_login=pd.read_csv("fitbitSubjectTokens_workingOnly.csv")
for i in range (fitbit_login.shape[0]):
    ID=fitbit_login['User_ID'][i]
    client_id=fitbit_login['Client_ID'][i]
    client_secret=fitbit_login['Client_secret'][i]
    participant_id=fitbit_login['Participant'][i]
    token=fitbit_login['Access_Token'][i]
    start_per1=fitbit_login['Start_per1'][i]
    start_per2=fitbit_login['Start_per2'][i]
    start_per3=fitbit_login['Start_per3'][i]
    end_per1=fitbit_login['End_per1'][i]
    end_per2=fitbit_login['End_per2'][i]
    end_per3=fitbit_login['End_per3'][i]
    treatment=fitbit_login['treatment'][i]
    if not start_per1 is np.NaN:
        start_date=convert_string_to_date(start_per1)
        end_date=datetime.date.today()
        fitbit_object=fitbit.Fitbit(client_id, client_secret,token,token)
        for j in range(len(activity)):
            activity_type=activity[j]
            if activity_type=='calories'or activity_type=='steps' or activity_type=='distance' or activity_type=='heart':
                df=get_daily_data(fitbit_object,'activities/'+activity_type,ID,start_date,end_date,treatment,ID)
                if activity_type=='calories':
                    df_calories_till_today = df_calories_till_today.append(df, ignore_index=True)
                elif activity_type=='steps':
                    df_steps_till_today = df_steps_till_today.append(df, ignore_index=True)
                elif activity_type=='distance':
                    df_distance_till_today = df_distance_till_today.append(df, ignore_index=True)
                else:
                    df_heart_till_today = df_heart_till_today.append(df, ignore_index=True)
            else:
                if activity_type=='sleep':
                    print("do nothing in sleep")
                    #df=get_daily_data(fitbit_object,activity_type,ID,start_date,end_date)
                else:
                    if activity_type=='bmi' or activity_type=='fat' or activity_type=='weight':
                        df=get_daily_data(fitbit_object,'body/'+activity_type,ID,start_date,end_date,treatment,ID)
                        if activity_type=='bmi':
                            df_bmi_till_today = df_bmi_till_today.append(df, ignore_index=True)
                        elif activity_type=='fat':
                            df_fat_till_today= df_fat_till_today.append(df, ignore_index=True)
                        else:
                            df_weight_till_today = df_weight_till_today.append(df, ignore_index=True)
                    else:
                        print("do nothing in activities tracker")
                        df=get_daily_data(fitbit_object,'activities/tracker/'+activity_type,ID,start_date,end_date,treatment,ID)
                        if activity_type=='minutesSedentary':
                            df_minutesSedentary_till_today = df_minutesSedentary.append(df, ignore_index=True)
                        elif activity_type=='minutesLightlyActive':
                            df_minutesLightlyActive_till_today = df_minutesLightlyActive.append(df, ignore_index=True)
                        elif activity_type=='minutesFairlyActive':
                            df_minutesFairlyActive_till_today = df_minutesFairlyActive.append(df, ignore_index=True)                              
                        elif activity_type=='minutesVeryActive':    
                            df_minutesVeryActive_till_today = df_minutesVeryActive.append(df, ignore_index=True)                          
                        #else:
                            #df_activityCalories = df_activityCalories.append(df, ignore_index=True)'''
df_minutesSedentary_till_today.to_csv('MS_till_today.csv', header=True, index = False)
df_minutesLightlyActive_till_today.to_csv('MLA_till_today.csv', header=True, index = False)
df_minutesFairlyActive_till_today.to_csv('MFA_till_today.csv', header=True, index = False)
df_minutesVeryActive_till_today.to_csv('MVA_till_today.csv', header=True, index = False)
#df_activityCalories.to_csv('AC_till_today.csv', header=True, index = False)

df_calories_till_today.to_csv('Calories.csv', header=True, index = False)  
df_steps_till_today.to_csv('Steps.csv', header=True, index = False)
df_distance_till_today.to_csv('Distance.csv', header=True, index = False)
#df_heart_till_today.to_csv('Heart.csv', header=True, index = False)
df_bmi_till_today.to_csv('Bmi.csv', header=True, index = False)
df_fat_till_today.to_csv('Fat.csv', header=True, index = False)
df_weight_till_today.to_csv('Weight.csv', header=True, index = False)


"df_calories_3month.to_csv('Calories_3month.csv', header=True, index = False)  \ndf_steps_3month.to_csv('Steps_3month.csv', header=True, index = False)\ndf_distance_3month.to_csv('Distance_3month.csv', header=True, index = False)\ndf_heart_3month.to_csv('Heart_3month.csv', header=True, index = False)\ndf_bmi_3month.to_csv('Bmi_3month.csv', header=True, index = False)\ndf_fat_3month.to_csv('Fat_3month.csv', header=True, index = False)\ndf_weight_3month.to_csv('Weight_3month.csv', header=True, index = False)\n\ndf_minutesSedentary_3month.to_csv('MS_3month.csv', header=True, index = False)\ndf_minutesLightlyActive_3month.to_csv('MLA_3month.csv', header=True, index = False)\ndf_minutesFairlyActive_3month.to_csv('MFA_3month.csv', header=True, index = False)\ndf_minutesVeryActive_3month.to_csv('MVA_3month.csv', header=True, index = False)\n#df_activityCalories_3month.to_csv('AC_3month.csv', header=True, index = False)\n"